###1. Análise Exploratória

In [0]:
#Ler a tabela da camada Gold, alocar em Spark Dataframe e criar uma View Temporária
MOUNT_CLEANED_VARIAVEIS = '/mnt/databricks-felipeyoshimoto/gold/iota/variaveis/'

df_pop_total = spark.read.parquet(MOUNT_CLEANED_VARIAVEIS + 'ultima_variavel_populacao_total')
df_pop_total.createOrReplaceTempView("VIEW_VARIAVEIS_POPULACAO_TOTAL")

df_pop_homens = spark.read.parquet(MOUNT_CLEANED_VARIAVEIS + 'ultima_variavel_populacao_de_homens')
df_pop_homens.createOrReplaceTempView("VIEW_VARIAVEIS_POPULACAO_DE_HOMENS")      

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW VIEW_POPULACAO AS (
  SELECT
    pop.city_id,
    pop.city_name,
    CAST(pop.value AS int) as populacao_total,
    CAST(hom.value AS int) as populacao_homens
  FROM VIEW_VARIAVEIS_POPULACAO_TOTAL pop
  INNER JOIN VIEW_VARIAVEIS_POPULACAO_DE_HOMENS hom
    ON pop.city_id = hom.city_id
)

In [0]:
%sql

SELECT * FROM VIEW_POPULACAO

city_id,city_name,populacao_total,populacao_homens
1,São Paulo,10865573,5323385
135,Belém,1425923,659008
180,Goiânia,1393579,620857
342,Ilhéus,184236,89440
385,Guadalajara,4434878,2171514
398,Brasília,2786684,1228880
407,Floridablanca,263908,125292
408,Girón,170771,84957
409,Piedecuesta,142483,69343
412,Quito,2239191,48


Output can only be rendered in Databricks

###2. Modelo de Regressão Linear Simples com PySpark MLlib

In [0]:
df_regressao = spark.sql('SELECT * FROM VIEW_POPULACAO')

###2.1. Importar a biblioteca PySpark MLlib com as funções necessárias para o modelo

In [0]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler

###2.2. Assumindo que os dados estão carregados no Spark Dataframe df_refressao, e a variável independente é a coluna chamada "populacao_total", e a variável dependente é a coluna chamada "populacao_homens", precisamos criar um vector assembler para combinar variáveis independentes em um vetor coluna único.

In [0]:
vectorAssembler = VectorAssembler(inputCols=["populacao_total"], outputCol="features")
data = vectorAssembler.transform(df_regressao).select("features", "populacao_homens")

###2.3. Split dos dados entre treino e teste

In [0]:
train_data, test_data = data.randomSplit([0.8, 0.2], seed=123)

###2.4. Criar um objeto do tipo LinearRegression e chamar o método fit no conjunto de dados de treino

In [0]:
# Create a LinearRegression object and fit the model on the training data
lr = LinearRegression(featuresCol = 'features', labelCol='populacao_homens')
lr_model = lr.fit(train_data)

In [0]:
# Print the coefficients and intercept of the model
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Coefficients: [0.48926566733229027]
Intercept: -77225.30878559024


###2.5. Utilizar o modelo para predição no conjunto de dados de teste

In [0]:
lr_predictions = lr_model.transform(test_data)

In [0]:
lr_predictions.show(5)

+-----------+----------------+------------------+
|   features|populacao_homens|        prediction|
+-----------+----------------+------------------+
| [184236.0]|           89440|12915.040701041595|
| [526827.0]|          252758|180533.05493807822|
|[1393579.0]|          620857| 604605.0506296755|
|[1425923.0]|          659008| 620429.8593738711|
|[4434878.0]|         2171514|2092608.2354217023|
+-----------+----------------+------------------+

